# Confusion matrix

## Import required Libraries:

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import script.ImageProcessing as IP
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# Avoid out of memory errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

## Load Your Model and Data:
Load your trained model and the data you want to evaluate. Make sure you preprocess the data in the same way you did during training.

In [ ]:
DIR = "D:\\Github Public\\DATASET MANAGER\\CONFUSION"
datapath_list = [os.path.join(root,image) for root, _, files in os.walk(DIR, topdown=True) for image in files]
datapath_list = [path for path in datapath_list if any(ext in path for ext in (".png", ".jpg"))]
datapath_list = [path for path in datapath_list if "TESTING" not in path]
datapath_list = [(path, path.split('\\')[-2]) for path in datapath_list]

In [ ]:
imagepath_list = [path for path, _ in datapath_list]
label_list = [label for _, label in datapath_list]

class_list = sorted(list(set(label_list)))


In [ ]:
MODEL = load_model('./export model/Res2Net50_224x224_2Class_fp16 (ILLUST VS MANGA).h5')  # Load your trained Keras model
IMAGE_SIZE = MODEL.layers[0].input_shape[0][1]
PROG_BAR = tf.keras.utils.Progbar(len(imagepath_list))

## Generate Predictions:
Use your model to predict the classes for the input data.

In [ ]:
def predict(image_path):
    image = IP.image_resize_square(image_path, IMAGE_SIZE, aspect_ratio=True, padding=True)
    image = np.expand_dims(image.numpy(), axis=0)
    PROG_BAR.add(1)
    #progress_bar.update(idx+1)
    return MODEL.predict(image)

In [ ]:
max_threads = os.cpu_count()  # Maximum number of concurrent threads
with ThreadPoolExecutor(max_threads) as executor:
    predictions_list = list(executor.map(predict, imagepath_list))

# Concatenate the results
predicted_class = np.concatenate(predictions_list)
predicted_class = np.argmax(predicted_class, axis=1)
predicted_class = [class_list[idx] for idx in predicted_class]

## Calculate the Confusion Matrix:
Create the confusion matrix using the true labels and predicted labels.

In [ ]:
# Assuming 'y_test' contains the true labels
confusion_mtx = confusion_matrix(label_list, predicted_class)


## Display the Confusion Matrix:
You can visualize the confusion matrix using a heatmap for better interpretation.

In [ ]:
print("Confusion Matrix:")
print(confusion_mtx)

# Plotting the confusion matrix as a heatmap
plt.figure(figsize=(8,6))
plt.imshow(confusion_mtx, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

# Adding axis labels
plt.xticks(np.arange(len(class_list)), class_list, rotation=45)
plt.yticks(np.arange(len(class_list)), class_list)

# Adding annotations (numbers) in the cells
for i in range(len(class_list)):
    for j in range(len(class_list)):
        plt.text(j, i, str(confusion_mtx[i, j]), ha='center', va='center', color='white')

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()